In [ ]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense
import tensorflow as tf
import random
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
random.seed(datetime.now())

In [ ]:
%run Grid.ipynb
%run ActorCriticAgent.ipynb

In [ ]:
NUM_EPISODES = 20000
MAX_MOVES = 100
REPLAY_INTERVAL = 50
LOG_FILE = "replays.txt"

log_file = open(LOG_FILE, "a")
print("Starting New Run", file=log_file)

agent = Agent(0.01, 0.01, memory_size=5000)
rewards = []
epsilon = 0 # No forced randomness
epsilon_decay = 0.995

for e in range(NUM_EPISODES):
    print("\r Episode : " + str(e), end = "")
    train = True if (e // REPLAY_INTERVAL > 0 and e % REPLAY_INTERVAL == 0) else False
    width = random.randint(5, 25)
    height = width #random.randint(1, 100)
    MAX_MOVES = 2.5 * (width + height)
    grid = Grid(width, height, random.randint(0, height - 1), random.randint(0, width - 1), 
                random.randint(0, height - 1), random.randint(0, width - 1), num_obstacles=random.randint(0, 0))
    gs0, gameOver = grid.getState()
    moves = 0
    cum_reward = 0
    
    if(train):
        print("New Game : %s" % (grid.getState2()), file = log_file)
        print("Episode : " + str(e), file = log_file)
        print(grid.getObstacles(), file = log_file)
    
    while(gameOver == False and moves < MAX_MOVES):
        moves += 1
        gs0, _ = grid.getState()
        action = agent.choose_action(gs0, ep=epsilon)
        gs1, reward = grid.move(action)
        gameOver = gs1[1]
        gs1 = gs1[0]
        agent.stateTransition(gs0, action, reward, gs1, gameOver)
        cum_reward += reward
        if(train):
            print(grid.getState2(), file = log_file)
    
    rewards.append(cum_reward)
    epsilon *= epsilon_decay
    if(train):
        # break
        sns.lineplot(x = range(len(rewards)), y = rewards)
        plt.savefig(f"figures/rewards.png")
        plt.clf()
        if(gameOver):
            print("GAME OVER!", file = log_file)
        log_file.flush()
        agent.learn()

log_file.close()

In [ ]:
log_file.close()